<a href="https://colab.research.google.com/github/BioMolDynamics/DeepPurpose-MD-Discovery/blob/main/Pipeline_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This notebook is a demo for running the pipeline covering DeepPurpose + AutoDock Vina + OpenMM. In this demo, we utilize SARS-CoV-2 sgRNA SL1-5 region and 3',5'-Dimethoxy-4-stilbenol as example.


--- Runtime recommendation ---

DeepPurpose and MD simulation require A100 GPU.

Autodock can run by a smaller GPU.

## Establish environment

In [ ]:
# Step 1: Install Conda + Conda dependencies
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/download/24.11.2-1_colab/Miniforge3-colab-24.11.2-1_colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:07
🔁 Restarting kernel...


### Wait for crash..

In [ ]:
import condacolab
condacolab.check()

✨🍰✨ Everything looks OK!


## Install dependencies

In [ ]:
# Clone the GitHub repo
!git clone https://github.com/BioMolDynamics/DeepPurpose-MD-Discovery.git

Cloning into 'DeepPurpose-MD-Discovery'...
remote: Enumerating objects: 91, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 91 (delta 45), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (91/91), 990.12 KiB | 3.78 MiB/s, done.
Resolving deltas: 100% (45/45), done.


In [ ]:
%cd DeepPurpose-MD-Discovery

/content/DeepPurpose-MD-Discovery


In [ ]:
!wget https://github.com/ccsb-scripps/AutoDock-Vina/releases/download/v1.2.5/vina_1.2.5_linux_x86_64
!chmod +x vina_1.2.5_linux_x86_64
!./vina_1.2.5_linux_x86_64 --version

--2025-08-07 08:17:43--  https://github.com/ccsb-scripps/AutoDock-Vina/releases/download/v1.2.5/vina_1.2.5_linux_x86_64
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/258054635/b208f84f-df05-4575-9991-2190698c7914?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-08-07T09%3A02%3A45Z&rscd=attachment%3B+filename%3Dvina_1.2.5_linux_x86_64&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-08-07T08%3A01%3A54Z&ske=2025-08-07T09%3A02%3A45Z&sks=b&skv=2018-11-09&sig=KFIhf5mSQZ%2FfyVwCRLcLLq%2BxSN3pvz95AH143HBVEhQ%3D&jwt=eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc1NDU1NDk2MywibmJmIjoxNzU0NTU0NjYzLCJwYXRoIjoicmV

In [ ]:
!git clone https://github.com/BioMolDynamics/Deeppurpose

Cloning into 'Deeppurpose'...
remote: Enumerating objects: 77, done.
remote: Counting objects: 100% (77/77), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 77 (delta 18), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (77/77), 3.68 MiB | 9.53 MiB/s, done.
Resolving deltas: 100% (18/18), done.


In [ ]:
!mamba env create -f environment.yml

Channels:
 - conda-forge
 - defaults
Platform: linux-64
Solving environment: - \ | / - \ | done


==> WARNING: A newer version of conda exists. <==
    current version: 24.11.2
    latest version: 25.7.0

Please update conda by running

    $ conda update -n base -c conda-forge conda



libcufft-11.4.1.4    | 154.6 MB  | :   0% 0/1 [00:00<?, ?it/s]
mkl-2022.1.0         | 129.7 MB  | :   0% 0/1 [00:00<?, ?it/s]

ambertools-24.8      | 96.5 MB   | :   0% 0/1 [00:00<?, ?it/s]


cuda-nvrtc-12.9.86   | 64.1 MB   | :   0% 0/1 [00:00<?, ?it/s]



pillow-11.3.0        | 41.1 MB   | :   0% 0/1 [00:00<?, ?it/s]




python-3.11.13       | 29.2 MB   | :   0% 0/1 [00:00<?, ?it/s]





rdkit-2025.03.5      | 19.3 MB   | :   0% 0/1 [00:00<?, ?it/s]






parmed-4.3.0         | 18.5 MB   | :   0% 0/1 [00:00<?, ?it/s]







scipy-1.16.0         | 16.1 MB   | :   0% 0/1 [00:00<?, ?it/s]








pandas-2.3.1         | 14.7 MB   | :   0% 0/1 [00:00<?, ?it/s]









perl-5.32.1          | 

In [ ]:
# Install Deeppurpose except dependencies to avoid conflict
!conda run -n deeppurpose-md-env pip install --no-deps ./Deeppurpose

Processing ./Deeppurpose
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for DeepPurpose: filename=deeppurpose-0.1.5-py3-none-any.whl size=159547 sha256=4e648846da5ae60994e33fe716d4499a8b152f51227f6ed7e5bc8f62ba17641f
  Stored in directory: /tmp/pip-ephem-wheel-cache-fm0a6cvq/wheels/30/f3/f7/36224af4e00db770a5038a5c2a40ee2b45a227332d2f1e0110
Successfully built DeepPurpose

  DEPRECATION: Building 'DeepPurpose' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'DeepPurpose'. Discussion can be found at https://github.com/pypa/pip/issues/6334



In [ ]:
!conda run -n deeppurpose-md-env python scripts/install_optional.py

  Cloning https://github.com/bp-kelley/descriptastorus to /tmp/pip-req-build-raz9izib
  Resolved https://github.com/bp-kelley/descriptastorus to commit 9a190343bcd3cfd35142d378d952613bcac40797
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for descriptastorus: filename=descriptastorus-2.7.0.3-py3-none-any.whl size=1082955 sha256=9e5a47934615efecb9fbb0df1fbf54cfbecfe6d0d0e18c69b26195caded1e954
  Stored in directory: /tmp/pip-ephem-wheel-cache-ytvad4_l/wheels/a5/37/b0/bc8b04ad2a1d1f3e17a7bff134d2a7e63621914e0a01d983a6
Successfully built descriptastorus
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 52.0 MB/s  0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject

## Autodock

### Prepare ligand

In [ ]:
# Step 1: Ligand Preparation
ligand_smiles = "COC1=CC(=CC(=C1)/C=C/C2=CC=C(C=C2)O)OC"  # 3',5'-Dimethoxy-4-stilbenol, for example
!conda run -n deeppurpose-md-env python scripts/1_prepare_ligand.py "$ligand_smiles"

✅ PDB file written: ligand.pdb
✅ mol2 file created: ligand.mol2
✅ PDBQT file created: ligand.pdbqt

1 molecule converted
1 molecule converted



### Prepare protein / RNA

In [ ]:
# Step 2: Receptor Preparation
pdb_id = "7B62" # S1 NTD, for example of protein

# For protein
# !conda run -n deeppurpose-md-env python scripts/2_prepare_receptor.py --pdb-id 7B62 --skip-fix

# For RNA. Ensure to store receptor.pdb in our repo (DeepPurpose-MD-Discovery) before running this.
!conda run -n deeppurpose-md-env python scripts/2_prepare_receptor.py --rna --input-pdb receptor.pdb


Files written:
   receptor.json <-- parameterized receptor
  receptor.pdbqt <-- static (i.e., rigid) receptor input file
receptor.box.txt <-- Vina-style box dimension file
receptor.box.pdb <-- PDB file to visualize the grid box
🧬 [RNA MODE] Using uploaded PDB: receptor.pdb
🛠️ Running Meeko on: receptor_clean.pdb
✅ Receptor PDBQT prepared.

/usr/local/envs/deeppurpose-md-env/lib/python3.11/site-packages/prody/utilities/misctools.py:424: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
@> 6281 atoms and 1 coordinate set(s) were parsed in 0.05s.
No template matched for residue_key='A:2'
tried 4 templates for residue_key='A:2'excess_H_ok=False
U          heavy_miss=0 heavy_excess=0 H_excess=[] bond_miss=set() bond_excess={27, 13}
U3         heavy_miss=0 heavy_excess=0 H_excess

### Run docking

In [ ]:
# Step 3: Docking with AutoDock Vina

# For protein
# !conda run -n deeppurpose-md-env python scripts/3_docking_vina.py --use-residue-centroid

# For RNA (SL5 as example)
!conda run -n deeppurpose-md-env python scripts/3_docking_vina.py --rna --res-range 149-297

# For RNA (Full atom centroid)
# !conda run -n deeppurpose-md-env python scripts/3_docking_vina.py --rna

AutoDock Vina v1.2.5
#################################################################
# If you used AutoDock Vina in your work, please cite:          #
#                                                               #
# J. Eberhardt, D. Santos-Martins, A. F. Tillack, and S. Forli  #
# AutoDock Vina 1.2.0: New Docking Methods, Expanded Force      #
# Field, and Python Bindings, J. Chem. Inf. Model. (2021)       #
# DOI 10.1021/acs.jcim.1c00203                                  #
#                                                               #
# O. Trott, A. J. Olson,                                        #
# AutoDock Vina: improving the speed and accuracy of docking    #
# with a new scoring function, efficient optimization and       #
# multithreading, J. Comp. Chem. (2010)                         #
# DOI 10.1002/jcc.21334                                         #
#                                                               #
# Please see https://github.com/ccsb-scripps/AutoDock-V

## MD simulation with OpenMM

### Prepare receptor for MD simulation

In [ ]:
# For protein
# !conda run -n deeppurpose-md-env python scripts/3b_prepare_protein.py

# For RNA
!conda run -n deeppurpose-md-env python scripts/3b_prepare_protein.py --rna

🧬 [RNA MODE] Running PDBFixer gently (clear missing residues)...
✅ PDBFixer complete: receptor_fixed.pdb
🩺 Removing 5′ terminal phosphates from first residue...
✅ Terminal patching complete → receptor_patched.pdb
💧 Adding hydrogens with RNA forcefield...
🎉 Final RNA receptor ready: receptor_cleaned.pdb



### Prepare ligand for MD simulation

In [ ]:
# Step 4: Align Ligand Coordinates for MD
!conda run -n deeppurpose-md-env python scripts/4_align_ligand.py

✅ Loaded 20 docked, 35 full, 20 stripped atoms.
✅ Aligned ligand written to: aligned_ligand_fixed.pdb
✅ Fixed PDB saved as fixed_ligand.pdb
✅ Ligand exported to ligand.sdf for forcefield assignment.

1 molecule converted



### Run MD simulation! (2ns simulation takes approx. 60 mins)

In [ ]:
# Step 5: Run MD Simulation

# For protein
# !conda run -n deeppurpose-md-env python scripts/5_md_simulation.py

# For protein (Apo)
# !conda run -n deeppurpose-md-env python scripts/5_md_simulation.py --no-ligand

# For RNA (25,000 steps as example)
!conda run -n deeppurpose-md-env python scripts/5_md_simulation.py --rna --n-steps 25000

# For RNA (Apo)
# !conda run -n deeppurpose-md-env python scripts/5_md_simulation.py --rna --no-ligand --n-steps 25000

🧬 [RNA MODE] Using amber14/RNA.OL3 forcefield
✅ Ligand merged
✅ System ready for solvation (holo)
✅ Solvated system ready.
🔹 Energy Minimization...
🔹 NVT Equilibration (1 ps)...
#"Step","Potential Energy (kJ/mole)","Temperature (K)"
100,-647694.9519319641,173.85529851242808
200,-642513.1477570613,198.32579339260352
300,-638088.3260444878,216.64084216581182
400,-634752.4333467849,231.72568449647284
500,-631835.151074446,246.32615553703516
🔹 NPT Equilibration (5 ps)...
600,-629322.9096346267,256.1058354309259
700,-627903.7940340408,265.76225120329906
800,-626833.1275606521,274.9721404570225
900,-625209.6356783279,278.7413662784091
1000,-625037.6824427017,285.5447697892855
1100,-623483.2648541953,286.81143548352367
1200,-623027.6381486212,290.1481624223654
1300,-622838.1602665205,292.06362120947233
1400,-623360.659717204,296.08650847608345
1500,-622844.1029776027,297.82002583830644
1600,-622923.3606083593,297.65760371983225
1700,-623665.3791306494,301.0361449526802
1800,-623374.2933152197

### Trajectory analyses

In [ ]:
# You may need this in *every* new Colab session
!pip install -q mdtraj biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 141.3 MB/s eta 0:00:00


In [ ]:
#@title [Supplementary for aromatic ring definition] Preview ligand atom names and indices

import mdtraj as md
ligand_pdb = "final_structure.pdb"
ligand_resname = "UNK"  # Update as needed

top = md.load_pdb(ligand_pdb).topology
print(f"Atoms in ligand residue ({ligand_resname}):")
for atom in top.atoms:
    if atom.residue.name.strip() == ligand_resname:
        print(f"  {atom.name.strip():<8} index={atom.index}")


Atoms in ligand residue (UNK):
  C1x      index=9459
  O1x      index=9460
  C2x      index=9461
  C3x      index=9462
  C4x      index=9463
  C5x      index=9464
  C6x      index=9465
  C7x      index=9466
  C8x      index=9467
  C9x      index=9468
  C10x     index=9469
  C11x     index=9470
  C12x     index=9471
  C13x     index=9472
  C14x     index=9473
  C15x     index=9474
  O2x      index=9475
  O3x      index=9476
  C16x     index=9477
  H1x      index=9478
  H2x      index=9479
  H3x      index=9480
  H4x      index=9481
  H5x      index=9482
  H6x      index=9483
  H7x      index=9484
  H8x      index=9485
  H9x      index=9486
  H10x     index=9487
  H11x     index=9488
  H12x     index=9489
  H13x     index=9490
  H14x     index=9491
  H15x     index=9492
  H16x     index=9493


In [ ]:
#@title 🚩 [Supplementary] Ligand π–π Ring Atom Definition (edit here before running π–π stacking analysis)

# List all atom names for each aromatic ring in your ligand
# Example: Two rings for a stilbene-like compound (replace with your atom names)
ligand_ring_defs = [
    ["C2x", "C3x", "C4x", "C5x", "C6x", "C7x"],      # Ring 1
    ["C15x", "C10x", "C11x", "C12x", "C13x", "C14x"]  # Ring 2
]

print("Ligand rings to be used for π–π stacking:")
for i, ring in enumerate(ligand_ring_defs):
    print(f" Ring {i+1}: {ring}")


Ligand rings to be used for π–π stacking:
 Ring 1: ['C2x', 'C3x', 'C4x', 'C5x', 'C6x', 'C7x']
 Ring 2: ['C15x', 'C10x', 'C11x', 'C12x', 'C13x', 'C14x']


In [ ]:
#@title [backend] Run to setup: Imports and Utility Functions

import jax.numpy as jnp
import numpy as np
import matplotlib.pyplot as plt
import mdtraj as md
from tqdm import tqdm
import seaborn as sns
import pandas as pd
import builtins
from collections import defaultdict
from sklearn.decomposition import PCA
from scipy.constants import Boltzmann

# Helper: Get ligand atom indices
def get_ligand_atoms(traj, ligand_resname="UNK"):
    return [atom.index for atom in traj.topology.atoms if atom.residue.name.strip() == ligand_resname]

# 1. Global RMSD (Overlay: holo vs apo)
def run_rmsd_overlay(traj_apo_file, top_apo_file, traj_holo_file, top_holo_file, out_prefix=""):
    traj_apo = md.load_dcd(traj_apo_file, top=top_apo_file)
    traj_holo = md.load_dcd(traj_holo_file, top=top_holo_file)
    exclude = {"HOH", "NA", "CL"}
    valid_atoms_apo = [i for i, a in enumerate(traj_apo.topology.atoms) if a.residue.name.strip() not in exclude]
    valid_atoms_holo = [i for i, a in enumerate(traj_holo.topology.atoms) if a.residue.name.strip() not in exclude]
    coords_apo = jnp.array(traj_apo.xyz)[:, valid_atoms_apo, :]
    coords_holo = jnp.array(traj_holo.xyz)[:, valid_atoms_holo, :]
    ref_xyz_apo = coords_apo[0]
    ref_xyz_holo = coords_holo[0]
    def compute_rmsd(coords, ref):
        return jnp.sqrt(jnp.mean(jnp.square(coords - ref), axis=(1,2)))
    rmsd_apo = compute_rmsd(coords_apo, ref_xyz_apo)
    rmsd_holo = compute_rmsd(coords_holo, ref_xyz_holo)
    plt.figure(figsize=(10, 4))
    plt.plot(rmsd_apo, label="Apo (no ligand)")
    plt.plot(rmsd_holo, label="Ligand-bound")
    plt.xlabel("Frame")
    plt.ylabel("RMSD (nm)")
    plt.title("Global RMSD: Apo vs Ligand-bound (JAX)")
    plt.legend()
    plt.tight_layout()
    fname = f"{out_prefix}rmsd_overlay.png"
    plt.savefig(fname)
    print(f"✅ RMSD overlay saved as {fname}")

# 2. Global RMSF (Overlay)
def run_rmsf_overlay(traj_apo_file, top_apo_file, traj_holo_file, top_holo_file, out_prefix=""):
    traj_apo = md.load_dcd(traj_apo_file, top=top_apo_file)
    traj_holo = md.load_dcd(traj_holo_file, top=top_holo_file)
    exclude = {"HOH", "NA", "CL"}
    valid_atoms_apo = [i for i, a in enumerate(traj_apo.topology.atoms) if a.residue.name.strip() not in exclude]
    valid_atoms_holo = [i for i, a in enumerate(traj_holo.topology.atoms) if a.residue.name.strip() not in exclude]
    coords_apo = jnp.array(traj_apo.xyz)[:, valid_atoms_apo, :]
    coords_holo = jnp.array(traj_holo.xyz)[:, valid_atoms_holo, :]
    avg_xyz_apo = jnp.mean(coords_apo, axis=0)
    avg_xyz_holo = jnp.mean(coords_holo, axis=0)
    rmsf_apo = jnp.sqrt(jnp.mean(jnp.sum((coords_apo - avg_xyz_apo[None, :, :])**2, axis=2), axis=0))
    rmsf_holo = jnp.sqrt(jnp.mean(jnp.sum((coords_holo - avg_xyz_holo[None, :, :])**2, axis=2), axis=0))
    plt.figure(figsize=(10, 4))
    plt.plot(rmsf_apo, label="Apo (no ligand)")
    plt.plot(rmsf_holo, label="Ligand-bound")
    plt.xlabel("Atom Index")
    plt.ylabel("RMSF (nm)")
    plt.title("Global RMSF: Apo vs Ligand-bound (JAX)")
    plt.legend()
    plt.tight_layout()
    fname = f"{out_prefix}rmsf_overlay.png"
    plt.savefig(fname)
    print(f"✅ RMSF overlay saved as {fname}")

# 3. Ligand RMSD (holo only)
def run_ligand_rmsd(traj_file, top_file, ligand_resname="UNK", out_prefix=""):
    traj = md.load_dcd(traj_file, top=top_file)
    ligand_atoms = get_ligand_atoms(traj, ligand_resname)
    if not ligand_atoms:
        print("❗ No ligand atoms found; skipping ligand RMSD.")
        return
    ligand_rmsd = md.rmsd(traj, traj, frame=0, atom_indices=ligand_atoms)
    plt.figure(figsize=(8, 5))
    plt.plot(traj.time / 1000, ligand_rmsd, label="Ligand RMSD (nm)")
    plt.xlabel("Time (ns)")
    plt.ylabel("RMSD (nm)")
    plt.title("Ligand RMSD Over Time")
    plt.legend()
    plt.grid()
    plt.tight_layout()
    fname = f"{out_prefix}ligand_rmsd.png"
    plt.savefig(fname)
    print(f"✅ Ligand RMSD plot saved as {fname}")

# 4. Hydration Shell (holo only, JAX-accelerated)
def run_hydration_shell(traj_file, top_file, ligand_resname="UNK", out_prefix="", dist_cutoff=0.35):
    traj = md.load_dcd(traj_file, top=top_file)
    ligand_atoms = get_ligand_atoms(traj, ligand_resname)
    water_atoms = traj.topology.select("water")
    # Compute distances (JAX)
    coords = jnp.array(traj.xyz)
    lig_xyz = coords[:, ligand_atoms, :]
    wat_xyz = coords[:, water_atoms, :]
    # Pairwise distances: (frames, n_lig, n_wat, 3)
    diff = lig_xyz[:, :, None, :] - wat_xyz[:, None, :, :]
    dists = jnp.linalg.norm(diff, axis=-1)  # (frames, n_lig, n_wat)
    min_dists = jnp.min(dists, axis=1)      # (frames, n_wat)
    hydration_counts = jnp.sum(min_dists < dist_cutoff, axis=1)
    plt.figure(figsize=(8, 5))
    plt.plot(np.arange(traj.n_frames), hydration_counts, label="Hydration Count (Water Bridges)")
    plt.xlabel("Frame")
    plt.ylabel("Number of Water Molecules")
    plt.title("Hydration Shell Around Ligand Over Time")
    plt.legend()
    plt.grid()
    plt.tight_layout()
    fname = f"{out_prefix}hydration_shell.png"
    plt.savefig(fname)
    print(f"✅ Hydration shell plot saved as {fname}")

# 5. Contact Analysis (holo only, but can extend)
def run_contact_analysis_rna(traj_file, top_file, ligand_resname="UNK", out_prefix=""):
    import mdtraj as md
    import jax.numpy as jnp
    from tqdm import tqdm
    from collections import defaultdict
    import pandas as pd
    import builtins

    traj = md.load_dcd(traj_file, top=top_file)
    coords = jnp.array(traj.xyz)
    rna_base_names = {"A", "U", "G", "C", "DA", "DG", "DT", "DC", "DU"}

    ligand_atoms = [atom.index for atom in traj.topology.atoms if atom.residue.name.strip() == ligand_resname]
    rna_atoms = [atom.index for atom in traj.topology.atoms if atom.residue.name.strip() in rna_base_names]

    atom_to_res_obj = {atom.index: atom.residue for atom in traj.topology.atoms if atom.index in rna_atoms}
    atom_to_res_label = {atom.index: f"{atom.residue.name} {atom.residue.resSeq}" for atom in traj.topology.atoms if atom.index in rna_atoms}

    cutoff = 0.4  # nm = 4 Å
    residue_contact_counts = defaultdict(int)
    residue_frame_hits = defaultdict(set)
    residue_unique_labels = defaultdict(set)

    for frame_idx in tqdm(range(coords.shape[0]), desc="Processing frames (contact)"):
        lig_xyz = coords[frame_idx, ligand_atoms]
        rna_xyz = coords[frame_idx, rna_atoms]
        diff = lig_xyz[:, None, :] - rna_xyz[None, :, :]
        dists = jnp.linalg.norm(diff, axis=-1)
        contacts = jnp.argwhere(dists < cutoff)
        for _, rna_idx in contacts:
            rna_index = int(rna_atoms[int(rna_idx)])
            res_obj = atom_to_res_obj[rna_index]
            res_label = atom_to_res_label[rna_index]
            residue_contact_counts[res_obj] += 1
            residue_frame_hits[res_obj].add(frame_idx)
            residue_unique_labels[res_label].add(frame_idx)

    total_contacts = builtins.sum(residue_contact_counts.values())
    interaction_frequencies = {
        f"{res}": (count / total_contacts) * 100 for res, count in residue_contact_counts.items()
    }
    residence_times = {
        f"{res}": (len(frames) / traj.n_frames) * 100 for res, frames in residue_frame_hits.items()
    }

    residue_frame_counts = {label: len(frames) for label, frames in residue_unique_labels.items()}
    sorted_counts = sorted(residue_frame_counts.items(), key=lambda x: x[1], reverse=True)

    df = pd.DataFrame({
        "Residue": list(interaction_frequencies.keys()),
        "Frequency (%)": list(interaction_frequencies.values()),
        "Residence Time (%)": [residence_times[res] for res in interaction_frequencies]
    }).sort_values(by="Frequency (%)", ascending=False).reset_index(drop=True)

    # === Output (text and csv)
    summary_txt = "\n🔹 Residue-Ligand Interaction Summary:\n" + df.to_string(index=False)
    summary_txt += "\n\n🔹 Most Frequent RNA Residue Contacts with Ligand (unique frames):\n"
    for res, count in sorted_counts[:20]:
        summary_txt += f"{res}: {count} frames\n"
    print(summary_txt)
    with open(f"{out_prefix}contact_summary.txt", "w") as f:
        f.write(summary_txt)
    df.to_csv(f"{out_prefix}contact_summary.csv", index=False)

    # === Bar plot
    plt.figure(figsize=(12, 4))
    plt.bar([x[0] for x in sorted_counts[:20]], [x[1] for x in sorted_counts[:20]])
    plt.xticks(rotation=90)
    plt.xlabel("RNA Residue")
    plt.ylabel("Unique Frames")
    plt.title("Top 20 Residue-Ligand Contacts (unique frames)")
    plt.tight_layout()
    plt.savefig(f"{out_prefix}contact_barplot.png")
    plt.show()
    print(f"✅ Contact summary: text, csv, and plot saved (prefix {out_prefix})")

def run_contact_analysis_protein(traj_file, top_file, ligand_resname="UNK", out_prefix=""):
    import mdtraj as md
    import jax.numpy as jnp
    from tqdm import tqdm
    from collections import defaultdict
    import pandas as pd
    import builtins

    traj = md.load_dcd(traj_file, top=top_file)
    coords = jnp.array(traj.xyz)

    # === Protein residue atom selection ===
    protein_resnames = set([res.name for res in traj.topology.residues if res.name not in {"HOH", ligand_resname}])
    ligand_atoms = [atom.index for atom in traj.topology.atoms if atom.residue.name.strip() == ligand_resname]
    protein_atoms = [atom.index for atom in traj.topology.atoms if atom.residue.name.strip() in protein_resnames]

    atom_to_res_obj = {atom.index: atom.residue for atom in traj.topology.atoms if atom.index in protein_atoms}
    atom_to_res_label = {atom.index: f"{atom.residue.name} {atom.residue.resSeq}" for atom in traj.topology.atoms if atom.index in protein_atoms}

    cutoff = 0.4  # nm = 4 Å
    residue_contact_counts = defaultdict(int)
    residue_frame_hits = defaultdict(set)
    residue_unique_labels = defaultdict(set)

    for frame_idx in tqdm(range(coords.shape[0]), desc="Processing frames (protein contact)"):
        lig_xyz = coords[frame_idx, ligand_atoms]
        prot_xyz = coords[frame_idx, protein_atoms]
        diff = lig_xyz[:, None, :] - prot_xyz[None, :, :]
        dists = jnp.linalg.norm(diff, axis=-1)
        contacts = jnp.argwhere(dists < cutoff)
        for _, prot_idx in contacts:
            prot_index = int(protein_atoms[int(prot_idx)])
            res_obj = atom_to_res_obj[prot_index]
            res_label = atom_to_res_label[prot_index]
            residue_contact_counts[res_obj] += 1
            residue_frame_hits[res_obj].add(frame_idx)
            residue_unique_labels[res_label].add(frame_idx)

    total_contacts = builtins.sum(residue_contact_counts.values())
    interaction_frequencies = {
        f"{res}": (count / total_contacts) * 100 for res, count in residue_contact_counts.items()
    }
    residence_times = {
        f"{res}": (len(frames) / traj.n_frames) * 100 for res, frames in residue_frame_hits.items()
    }

    residue_frame_counts = {label: len(frames) for label, frames in residue_unique_labels.items()}
    sorted_counts = sorted(residue_frame_counts.items(), key=lambda x: x[1], reverse=True)

    df = pd.DataFrame({
        "Residue": list(interaction_frequencies.keys()),
        "Frequency (%)": list(interaction_frequencies.values()),
        "Residence Time (%)": [residence_times[res] for res in interaction_frequencies]
    }).sort_values(by="Frequency (%)", ascending=False).reset_index(drop=True)

    summary_txt = "\n🔹 Protein Residue-Ligand Interaction Summary:\n" + df.to_string(index=False)
    summary_txt += "\n\n🔹 Most Frequent Protein Residue Contacts with Ligand (unique frames):\n"
    for res, count in sorted_counts[:20]:
        summary_txt += f"{res}: {count} frames\n"
    print(summary_txt)
    with open(f"{out_prefix}protein_contact_summary.txt", "w") as f:
        f.write(summary_txt)
    df.to_csv(f"{out_prefix}protein_contact_summary.csv", index=False)

    plt.figure(figsize=(12, 4))
    plt.bar([x[0] for x in sorted_counts[:20]], [x[1] for x in sorted_counts[:20]])
    plt.xticks(rotation=90)
    plt.xlabel("Protein Residue")
    plt.ylabel("Unique Frames")
    plt.title("Top 20 Protein Residue-Ligand Contacts (unique frames)")
    plt.tight_layout()
    plt.savefig(f"{out_prefix}protein_contact_barplot.png")
    plt.show()
    print(f"✅ Protein contact summary: text, csv, and plot saved (prefix {out_prefix})")


# 6. H-Bond Analysis (holo only, image heatmap)
def run_hbond_analysis_rna(traj_file, top_file, ligand_resname="UNK", out_prefix="", dist_cutoff=0.35, angle_cutoff=120.0):
    import mdtraj as md
    import jax.numpy as jnp
    from tqdm import tqdm
    from collections import defaultdict, Counter
    import pandas as pd
    import seaborn as sns
    import matplotlib.pyplot as plt
    import numpy as np

    traj = md.load_dcd(traj_file, top=top_file)
    coords = jnp.array(traj.xyz)
    atoms = list(traj.topology.atoms)
    bond_map = {i: [] for i in range(len(atoms))}
    for bond in traj.topology.bonds:
        bond_map[bond[0].index].append(bond[1].index)
        bond_map[bond[1].index].append(bond[0].index)
    acceptors = [a for a in atoms if a.element.symbol in ['N', 'O'] and a.residue.name.strip() != 'HOH']
    acceptor_indices_set = set(a.index for a in acceptors)
    triplets_np = []
    for h in atoms:
        if h.element.symbol != 'H' or h.residue.name.strip() == "HOH":
            continue
        bonded = bond_map[h.index]
        if not bonded:
            continue
        d_idx = bonded[0]
        for a_idx in acceptor_indices_set:
            if a_idx != d_idx:
                triplets_np.append((d_idx, h.index, a_idx))
    triplets_np = np.array(triplets_np, dtype=np.int32)
    triplets = jnp.array(triplets_np)
    donor_idxs = triplets[:, 0]
    hydrogen_idxs = triplets[:, 1]
    acceptor_idxs = triplets[:, 2]
    unk_atom_indices = jnp.array([a.index for a in atoms if a.residue.name.strip() == ligand_resname])
    involves_unk_mask = jnp.isin(donor_idxs, unk_atom_indices) | jnp.isin(acceptor_idxs, unk_atom_indices)

    h_bonds_found = []
    n_frames = coords.shape[0]
    for frame_idx in tqdm(range(n_frames), desc="Scanning H-bonds"):
        xyz = coords[frame_idx]
        d_xyz = xyz[donor_idxs]
        h_xyz = xyz[hydrogen_idxs]
        a_xyz = xyz[acceptor_idxs]
        vec_HA = h_xyz - a_xyz
        dist_HA = jnp.linalg.norm(vec_HA, axis=1)
        vec_DH = h_xyz - d_xyz
        norm_DH = vec_DH / jnp.linalg.norm(vec_DH, axis=1)[:, None]
        norm_HA = vec_HA / jnp.linalg.norm(vec_HA, axis=1)[:, None]
        cos_angle = jnp.sum(norm_DH * norm_HA, axis=1)
        angles = jnp.degrees(jnp.arccos(jnp.clip(cos_angle, -1.0, 1.0)))
        passed = jnp.where((dist_HA < dist_cutoff) & (angles > angle_cutoff) & involves_unk_mask)[0]
        for idx in passed:
            d = atoms[int(donor_idxs[idx])]
            a = atoms[int(acceptor_idxs[idx])]
            h_bonds_found.append((
                f"{d.residue.name} {d.residue.resSeq} ({d.name})",
                f"{a.residue.name} {a.residue.resSeq} ({a.name})"
            ))

    # Save all H-bonds to text file
    with open(f"{out_prefix}hbond_summary.txt", "w") as f:
        for donor, acceptor in h_bonds_found:
            f.write(f"{donor} → {acceptor}\n")
    print(f"✅ H-bond summary saved as {out_prefix}hbond_summary.txt")

    # Step 1: Frequencies (top 20)
    residue_counter = Counter()
    for donor, acceptor in h_bonds_found:
        for entry in (donor, acceptor):
            if entry.startswith("UNK"): continue
            res_label = entry.split('(')[0].strip()
            residue_counter[res_label] += 1
    df = pd.DataFrame(residue_counter.items(), columns=["Residue", "Count"]).sort_values("Count", ascending=False).reset_index(drop=True)
    plt.figure(figsize=(12, 6))
    plt.bar(df["Residue"][:20], df["Count"][:20], color='skyblue')
    plt.title("Top 20 Residues Involved in H-Bonds with UNK", fontsize=14)
    plt.xlabel("RNA Residue")
    plt.ylabel("H-Bond Count")
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.grid(axis='y', linestyle='--', alpha=0.6)
    plt.savefig(f"{out_prefix}hbond_top20_bar.png")
    plt.show()

    # Step 2: Donor vs Acceptor roles (top 20 stacked)
    donor_roles = Counter()
    acceptor_roles = Counter()
    for donor, acceptor in h_bonds_found:
        if donor.startswith("UNK"):
            res = acceptor.split('(')[0].strip()
            acceptor_roles[res] += 1
        elif acceptor.startswith("UNK"):
            res = donor.split('(')[0].strip()
            donor_roles[res] += 1
    residues = list(set(donor_roles) | set(acceptor_roles))
    df_roles = pd.DataFrame({
        "Residue": residues,
        "UNK as Donor": [donor_roles.get(r, 0) for r in residues],
        "UNK as Acceptor": [acceptor_roles.get(r, 0) for r in residues]
    }).sort_values(["UNK as Donor", "UNK as Acceptor"], ascending=False).head(20)
    df_roles.set_index("Residue").plot(kind="bar", stacked=True, figsize=(12, 6), colormap="viridis")
    plt.title("Top 20 H-Bond Partners (UNK as Donor vs Acceptor)", fontsize=14)
    plt.ylabel("Bond Count")
    plt.xlabel("RNA Residue")
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.grid(axis='y', linestyle='--', alpha=0.5)
    plt.savefig(f"{out_prefix}hbond_roles_top20_stacked.png")
    plt.show()

    # Step 3: Per-frame heatmap (top 20)
    framewise_counts = defaultdict(lambda: defaultdict(int))
    for frame_idx in tqdm(range(n_frames), desc="Building frame-wise heatmap"):
        xyz = coords[frame_idx]
        d_xyz = xyz[donor_idxs]
        h_xyz = xyz[hydrogen_idxs]
        a_xyz = xyz[acceptor_idxs]
        vec_HA = h_xyz - a_xyz
        dist_HA = jnp.linalg.norm(vec_HA, axis=1)
        vec_DH = h_xyz - d_xyz
        norm_DH = vec_DH / jnp.linalg.norm(vec_DH, axis=1)[:, None]
        norm_HA = vec_HA / jnp.linalg.norm(vec_HA, axis=1)[:, None]
        cos_angle = jnp.sum(norm_DH * norm_HA, axis=1)
        angles = jnp.degrees(jnp.arccos(jnp.clip(cos_angle, -1.0, 1.0)))
        passed = jnp.where((dist_HA < dist_cutoff) & (angles > angle_cutoff) & involves_unk_mask)[0]
        for idx in passed:
            d = atoms[int(donor_idxs[idx])]
            a = atoms[int(acceptor_idxs[idx])]
            for atom in [d, a]:
                if atom.residue.name.strip() == "UNK":
                    continue
                res_key = f"{atom.residue.name} {atom.residue.resSeq}"
                framewise_counts[frame_idx][res_key] += 1

    total_counts = defaultdict(int)
    for frame_dict in framewise_counts.values():
        for res, count in frame_dict.items():
            total_counts[res] += count
    top_residues = sorted(total_counts, key=total_counts.get, reverse=True)[:20]
    heatmap_df = pd.DataFrame(index=range(n_frames), columns=top_residues).fillna(0)
    for frame_idx, res_counts in framewise_counts.items():
        for res in top_residues:
            if res in res_counts:
                heatmap_df.at[frame_idx, res] = res_counts[res]
    plt.figure(figsize=(14, 6))
    sns.heatmap(heatmap_df.T, cmap="YlGnBu", cbar_kws={'label': 'H-Bond Count'})
    plt.title("H-Bond Time Series per Residue (Top 20)")
    plt.xlabel("Frame")
    plt.ylabel("RNA Residue")
    plt.tight_layout()
    plt.savefig(f"{out_prefix}hbond_heatmap.png")
    plt.show()

    print(f"✅ H-bond summary (txt), top-20 bar, role stacked, and per-frame heatmap plots saved (prefix {out_prefix})")

def run_hbond_analysis_protein(traj_file, top_file, ligand_resname="UNK", out_prefix="", dist_cutoff=0.35, angle_cutoff=120.0):
    import mdtraj as md
    import jax.numpy as jnp
    from tqdm import tqdm
    from collections import defaultdict, Counter
    import pandas as pd
    import seaborn as sns
    import matplotlib.pyplot as plt
    import numpy as np

    traj = md.load_dcd(traj_file, top=top_file)
    coords = jnp.array(traj.xyz)
    atoms = list(traj.topology.atoms)
    bond_map = {i: [] for i in range(len(atoms))}
    for bond in traj.topology.bonds:
        bond_map[bond[0].index].append(bond[1].index)
        bond_map[bond[1].index].append(bond[0].index)
    # Only protein atoms as donors/acceptors, not water or ligand
    protein_resnames = set([res.name for res in traj.topology.residues if res.name not in {"HOH", ligand_resname}])
    acceptors = [a for a in atoms if a.element.symbol in ['N', 'O'] and a.residue.name.strip() in protein_resnames]
    acceptor_indices_set = set(a.index for a in acceptors)
    triplets_np = []
    for h in atoms:
        if h.element.symbol != 'H' or h.residue.name.strip() not in protein_resnames:
            continue
        bonded = bond_map[h.index]
        if not bonded:
            continue
        d_idx = bonded[0]
        for a_idx in acceptor_indices_set:
            if a_idx != d_idx:
                triplets_np.append((d_idx, h.index, a_idx))
    triplets_np = np.array(triplets_np, dtype=np.int32)
    triplets = jnp.array(triplets_np)
    donor_idxs = triplets[:, 0]
    hydrogen_idxs = triplets[:, 1]
    acceptor_idxs = triplets[:, 2]
    ligand_atom_indices = jnp.array([a.index for a in atoms if a.residue.name.strip() == ligand_resname])
    involves_ligand_mask = jnp.isin(donor_idxs, ligand_atom_indices) | jnp.isin(acceptor_idxs, ligand_atom_indices)

    h_bonds_found = []
    n_frames = coords.shape[0]
    for frame_idx in tqdm(range(n_frames), desc="Scanning H-bonds (protein)"):
        xyz = coords[frame_idx]
        d_xyz = xyz[donor_idxs]
        h_xyz = xyz[hydrogen_idxs]
        a_xyz = xyz[acceptor_idxs]
        vec_HA = h_xyz - a_xyz
        dist_HA = jnp.linalg.norm(vec_HA, axis=1)
        vec_DH = h_xyz - d_xyz
        norm_DH = vec_DH / jnp.linalg.norm(vec_DH, axis=1)[:, None]
        norm_HA = vec_HA / jnp.linalg.norm(vec_HA, axis=1)[:, None]
        cos_angle = jnp.sum(norm_DH * norm_HA, axis=1)
        angles = jnp.degrees(jnp.arccos(jnp.clip(cos_angle, -1.0, 1.0)))
        passed = jnp.where((dist_HA < dist_cutoff) & (angles > angle_cutoff) & involves_ligand_mask)[0]
        for idx in passed:
            d = atoms[int(donor_idxs[idx])]
            a = atoms[int(acceptor_idxs[idx])]
            h_bonds_found.append((
                f"{d.residue.name} {d.residue.resSeq} ({d.name})",
                f"{a.residue.name} {a.residue.resSeq} ({a.name})"
            ))

    with open(f"{out_prefix}protein_hbond_summary.txt", "w") as f:
        for donor, acceptor in h_bonds_found:
            f.write(f"{donor} → {acceptor}\n")
    print(f"✅ Protein H-bond summary saved as {out_prefix}protein_hbond_summary.txt")

    residue_counter = Counter()
    for donor, acceptor in h_bonds_found:
        for entry in (donor, acceptor):
            if entry.startswith("UNK"): continue
            res_label = entry.split('(')[0].strip()
            residue_counter[res_label] += 1
    df = pd.DataFrame(residue_counter.items(), columns=["Residue", "Count"]).sort_values("Count", ascending=False).reset_index(drop=True)
    plt.figure(figsize=(12, 6))
    plt.bar(df["Residue"][:20], df["Count"][:20], color='orange')
    plt.title("Top 20 Protein Residues Involved in H-Bonds with Ligand", fontsize=14)
    plt.xlabel("Protein Residue")
    plt.ylabel("H-Bond Count")
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.grid(axis='y', linestyle='--', alpha=0.6)
    plt.savefig(f"{out_prefix}protein_hbond_top20_bar.png")
    plt.show()

    donor_roles = Counter()
    acceptor_roles = Counter()
    for donor, acceptor in h_bonds_found:
        if donor.startswith("UNK"):
            res = acceptor.split('(')[0].strip()
            acceptor_roles[res] += 1
        elif acceptor.startswith("UNK"):
            res = donor.split('(')[0].strip()
            donor_roles[res] += 1
    residues = list(set(donor_roles) | set(acceptor_roles))
    df_roles = pd.DataFrame({
        "Residue": residues,
        "Ligand as Donor": [donor_roles.get(r, 0) for r in residues],
        "Ligand as Acceptor": [acceptor_roles.get(r, 0) for r in residues]
    }).sort_values(["Ligand as Donor", "Ligand as Acceptor"], ascending=False).head(20)
    df_roles.set_index("Residue").plot(kind="bar", stacked=True, figsize=(12, 6), colormap="magma")
    plt.title("Top 20 H-Bond Partners (Ligand as Donor vs Acceptor)", fontsize=14)
    plt.ylabel("Bond Count")
    plt.xlabel("Protein Residue")
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.grid(axis='y', linestyle='--', alpha=0.5)
    plt.savefig(f"{out_prefix}protein_hbond_roles_top20_stacked.png")
    plt.show()

    # Per-frame heatmap (top 20)
    framewise_counts = defaultdict(lambda: defaultdict(int))
    for frame_idx in tqdm(range(n_frames), desc="Building protein H-bond heatmap"):
        xyz = coords[frame_idx]
        d_xyz = xyz[donor_idxs]
        h_xyz = xyz[hydrogen_idxs]
        a_xyz = xyz[acceptor_idxs]
        vec_HA = h_xyz - a_xyz
        dist_HA = jnp.linalg.norm(vec_HA, axis=1)
        vec_DH = h_xyz - d_xyz
        norm_DH = vec_DH / jnp.linalg.norm(vec_DH, axis=1)[:, None]
        norm_HA = vec_HA / jnp.linalg.norm(vec_HA, axis=1)[:, None]
        cos_angle = jnp.sum(norm_DH * norm_HA, axis=1)
        angles = jnp.degrees(jnp.arccos(jnp.clip(cos_angle, -1.0, 1.0)))
        passed = jnp.where((dist_HA < dist_cutoff) & (angles > angle_cutoff) & involves_ligand_mask)[0]
        for idx in passed:
            d = atoms[int(donor_idxs[idx])]
            a = atoms[int(acceptor_idxs[idx])]
            for atom in [d, a]:
                if atom.residue.name.strip() == ligand_resname:
                    continue
                res_key = f"{atom.residue.name} {atom.residue.resSeq}"
                framewise_counts[frame_idx][res_key] += 1

    total_counts = defaultdict(int)
    for frame_dict in framewise_counts.values():
        for res, count in frame_dict.items():
            total_counts[res] += count
    top_residues = sorted(total_counts, key=total_counts.get, reverse=True)[:20]
    heatmap_df = pd.DataFrame(index=range(n_frames), columns=top_residues).fillna(0)
    for frame_idx, res_counts in framewise_counts.items():
        for res in top_residues:
            if res in res_counts:
                heatmap_df.at[frame_idx, res] = res_counts[res]
    plt.figure(figsize=(14, 6))
    sns.heatmap(heatmap_df.T, cmap="YlOrBr", cbar_kws={'label': 'H-Bond Count'})
    plt.title("Protein H-Bond Time Series per Residue (Top 20)")
    plt.xlabel("Frame")
    plt.ylabel("Protein Residue")
    plt.tight_layout()
    plt.savefig(f"{out_prefix}protein_hbond_heatmap.png")
    plt.show()

    print(f"✅ Protein H-bond summary (txt), top-20 bar, role stacked, and per-frame heatmap plots saved (prefix {out_prefix})")


# 7. PCA & FEL (Overlay)
def run_pca_fel_overlay(traj_apo_file, top_apo_file, traj_holo_file, top_holo_file, out_prefix=""):
    traj_apo = md.load(traj_apo_file, top=top_apo_file)
    traj_holo = md.load(traj_holo_file, top=top_holo_file)
    # Select matching P atoms
    p_atoms_apo = traj_apo.top.select("name P")
    p_atoms_holo = traj_holo.top.select("name P")
    if len(p_atoms_apo) != len(p_atoms_holo):
        raise ValueError(f"Mismatched P atoms: apo={len(p_atoms_apo)}, holo={len(p_atoms_holo)}")
    traj_apo = traj_apo.atom_slice(p_atoms_apo)
    traj_holo = traj_holo.atom_slice(p_atoms_holo)
    traj_apo.superpose(traj_apo, 0)
    traj_holo.superpose(traj_apo, 0)
    xyz_apo = traj_apo.xyz.reshape(traj_apo.n_frames, -1)
    xyz_holo = traj_holo.xyz.reshape(traj_holo.n_frames, -1)
    pca = PCA(n_components=2)
    pca.fit(xyz_apo)
    pc_apo = pca.transform(xyz_apo)
    pc_holo = pca.transform(xyz_holo)
    # Overlay PCA plot
    plt.figure(figsize=(8, 5))
    plt.scatter(pc_apo[:, 0], pc_apo[:, 1], alpha=0.5, label="Apo (no ligand)", color="blue")
    plt.scatter(pc_holo[:, 0], pc_holo[:, 1], alpha=0.5, label="Ligand-bound", color="red")
    plt.xlabel(f"PC1 ({pca.explained_variance_ratio_[0]*100:.2f}% var)")
    plt.ylabel(f"PC2 ({pca.explained_variance_ratio_[1]*100:.2f}% var)")
    plt.title("PCA Projection: Ligand vs Apo (RNA)")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    fname = f"{out_prefix}pca_overlay.png"
    plt.savefig(fname)
    print(f"✅ PCA overlay plot saved as {fname}")
    # Free Energy Landscape (FEL) for both
    def compute_fel(pc1, pc2, bins=100):
        H, xedges, yedges = np.histogram2d(pc1, pc2, bins=bins, density=True)
        P = H.T
        P[P == 0] = 1e-12
        G = -Boltzmann * 310.15 * np.log(P)
        G -= np.min(G)
        return G, xedges, yedges
    G_apo, x_a, y_a = compute_fel(pc_apo[:, 0], pc_apo[:, 1])
    G_holo, x_h, y_h = compute_fel(pc_holo[:, 0], pc_holo[:, 1])
    # Overlay FEL
    X_a, Y_a = np.meshgrid(x_a[:-1], y_a[:-1])
    X_h, Y_h = np.meshgrid(x_h[:-1], y_h[:-1])
    plt.figure(figsize=(12, 5))
    plt.subplot(1,2,1)
    cp1 = plt.contourf(X_a, Y_a, G_apo, levels=30, cmap="Blues")
    plt.colorbar(cp1, label="Free Energy (J)")
    plt.title("FEL: Apo (No Ligand)")
    plt.xlabel("PC1"); plt.ylabel("PC2")
    plt.subplot(1,2,2)
    cp2 = plt.contourf(X_h, Y_h, G_holo, levels=30, cmap="Reds")
    plt.colorbar(cp2, label="Free Energy (J)")
    plt.title("FEL: Ligand-bound")
    plt.xlabel("PC1"); plt.ylabel("PC2")
    plt.tight_layout()
    fname_fel = f"{out_prefix}fel_overlay.png"
    plt.savefig(fname_fel)
    print(f"✅ FEL overlay plot saved as {fname_fel}")

# 8. WC checker (Overlay)

def run_wc_pair_checker_overlay(
    traj_apo_file, top_apo_file, traj_holo_file, top_holo_file, out_prefix=""
):
    """
    Overlays Watson–Crick base pair stability for both apo and holo MD runs.
    Saves plot as .png and prints number of pairs detected in each run.
    """
    import mdtraj as md
    import jax.numpy as jnp
    from tqdm import tqdm
    import matplotlib.pyplot as plt

    def get_pairs(traj, coords):
        topology = traj.topology
        atoms = list(topology.atoms)
        wc_pairs = {
            ("A", "U"): ("N1", "N3"),
            ("U", "A"): ("N3", "N1"),
            ("G", "C"): ("N1", "N3"),
            ("C", "G"): ("N3", "N1"),
        }
        min_seq_gap = 4
        atom_lookup = {
            (atom.residue.name.strip(), atom.residue.resSeq, atom.name): atom.index
            for atom in atoms
        }
        rna_residues = [
            res
            for res in topology.residues
            if res.name.strip() in {"A", "U", "G", "C"}
        ]
        pairs = []
        for i, res1 in enumerate(rna_residues):
            for j in range(i + 1, len(rna_residues)):
                res2 = rna_residues[j]
                if abs(res1.resSeq - res2.resSeq) < min_seq_gap:
                    continue
                pair_key = (res1.name.strip(), res2.name.strip())
                if pair_key not in wc_pairs:
                    continue
                atom1_name, atom2_name = wc_pairs[pair_key]
                idx1 = atom_lookup.get((res1.name.strip(), res1.resSeq, atom1_name))
                idx2 = atom_lookup.get((res2.name.strip(), res2.resSeq, atom2_name))
                if idx1 is None or idx2 is None:
                    continue
                dist = jnp.linalg.norm(coords[0, idx1] - coords[0, idx2])
                if dist < 0.35:
                    pairs.append((res1, idx1, res2, idx2))
        return pairs

    def pair_status_over_time(coords, pairs):
        framewise_status = []
        for frame in range(coords.shape[0]):
            count = 0
            for res1, idx1, res2, idx2 in pairs:
                dist = jnp.linalg.norm(coords[frame, idx1] - coords[frame, idx2])
                if dist < 0.35:
                    count += 1
            framewise_status.append(count)
        return framewise_status

    # Apo trajectory
    traj_apo = md.load_dcd(traj_apo_file, top=top_apo_file)
    coords_apo = jnp.array(traj_apo.xyz)
    pairs_apo = get_pairs(traj_apo, coords_apo)
    print(f"✅ [Apo] Detected {len(pairs_apo)} WC pairs in frame 0")
    wc_apo = pair_status_over_time(coords_apo, pairs_apo)

    # Holo trajectory
    traj_holo = md.load_dcd(traj_holo_file, top=top_holo_file)
    coords_holo = jnp.array(traj_holo.xyz)
    pairs_holo = get_pairs(traj_holo, coords_holo)
    print(f"✅ [Ligand] Detected {len(pairs_holo)} WC pairs in frame 0")
    wc_holo = pair_status_over_time(coords_holo, pairs_holo)

    # Plot
    plt.figure(figsize=(10, 4))
    plt.plot(wc_apo, label="Apo (No Ligand)", color="blue")
    plt.plot(wc_holo, label="Ligand-bound", color="red")
    plt.xlabel("Frame")
    plt.ylabel("Intact Watson–Crick Pairs")
    plt.title("WC Base Pair Stability Over Time")
    plt.grid()
    plt.legend()
    plt.tight_layout()
    fname = f"{out_prefix}wc_pair_stability_overlay.png"
    plt.savefig(fname)
    plt.show()
    print(f"📊 Overlay plot saved as {fname}")

# 9. Ligand–RNA Atom Contact Frequencies (Table, CSV, UI)

def run_ligand_rna_atom_contact_frequencies_safe(
    traj_file="production_md.dcd",
    top_file="final_structure.pdb",
    ligand_resname="UNK",
    out_prefix=""
):
    """
    Computes h-bonds between ligand and RNA, then tabulates RNA atom contacts.
    Self-contained and robust: does not depend on external/global h_bonds_found.
    """
    import mdtraj as md
    import jax.numpy as jnp
    import numpy as np
    import pandas as pd
    from collections import defaultdict
    import re
    from IPython.display import display
    from tqdm import tqdm

    # --- Load trajectory
    traj = md.load_dcd(traj_file, top=top_file)
    coords = jnp.array(traj.xyz)
    atoms = list(traj.topology.atoms)
    bond_map = {i: [] for i in range(len(atoms))}
    for bond in traj.topology.bonds:
        bond_map[bond[0].index].append(bond[1].index)
        bond_map[bond[1].index].append(bond[0].index)
    # Only RNA and ligand as donors/acceptors (no water)
    rna_bases = {"A", "U", "G", "C"}
    acceptors = [a for a in atoms if a.element.symbol in ['N', 'O'] and a.residue.name.strip() in rna_bases]
    acceptor_indices_set = set(a.index for a in acceptors)
    triplets_np = []
    for h in atoms:
        if h.element.symbol != 'H':
            continue
        if h.residue.name.strip() not in (rna_bases | {ligand_resname}):
            continue
        bonded = bond_map[h.index]
        if not bonded:
            continue
        d_idx = bonded[0]
        for a_idx in acceptor_indices_set:
            if a_idx != d_idx:
                triplets_np.append((d_idx, h.index, a_idx))
    triplets_np = np.array(triplets_np, dtype=np.int32)
    triplets = jnp.array(triplets_np)
    donor_idxs = triplets[:, 0]
    hydrogen_idxs = triplets[:, 1]
    acceptor_idxs = triplets[:, 2]
    ligand_atom_indices = jnp.array([a.index for a in atoms if a.residue.name.strip() == ligand_resname])
    involves_ligand_mask = jnp.isin(donor_idxs, ligand_atom_indices) | jnp.isin(acceptor_idxs, ligand_atom_indices)

    # --- Find h-bonds (RNA-ligand only)
    h_bonds_found = []
    dist_cutoff = 0.35  # nm
    angle_cutoff = 120.0  # degrees
    n_frames = coords.shape[0]
    for frame_idx in tqdm(range(n_frames), desc="Scanning ligand–RNA H-bonds"):
        xyz = coords[frame_idx]
        d_xyz = xyz[donor_idxs]
        h_xyz = xyz[hydrogen_idxs]
        a_xyz = xyz[acceptor_idxs]
        vec_HA = h_xyz - a_xyz
        dist_HA = jnp.linalg.norm(vec_HA, axis=1)
        vec_DH = h_xyz - d_xyz
        norm_DH = vec_DH / jnp.linalg.norm(vec_DH, axis=1)[:, None]
        norm_HA = vec_HA / jnp.linalg.norm(vec_HA, axis=1)[:, None]
        cos_angle = jnp.sum(norm_DH * norm_HA, axis=1)
        angles = jnp.degrees(jnp.arccos(jnp.clip(cos_angle, -1.0, 1.0)))
        passed = jnp.where((dist_HA < dist_cutoff) & (angles > angle_cutoff) & involves_ligand_mask)[0]
        for idx in passed:
            d = atoms[int(donor_idxs[idx])]
            a = atoms[int(acceptor_idxs[idx])]
            h_bonds_found.append((
                f"{d.residue.name} {d.residue.resSeq} ({d.name})",
                f"{a.residue.name} {a.residue.resSeq} ({a.name})"
            ))

    if not h_bonds_found:
        print("❗ No ligand–RNA H-bonds found. Nothing to tabulate.")
        return

    # --- Tally RNA atom contacts from h-bonds_found ---
    atom_contact_counts = defaultdict(int)
    for donor, acceptor in h_bonds_found:
        for entry in (donor, acceptor):
            if entry.startswith("UNK"):
                continue
            match = re.match(r"([AUGC])\s\d+\s\((\w+)\)", entry)
            if match:
                base, atom = match.groups()
                key = f"{base}-{atom}"
                atom_contact_counts[key] += 1

    df = pd.DataFrame(atom_contact_counts.items(), columns=["RNA Atom", "Contact Count"])
    df = df.sort_values(by="Contact Count", ascending=False).reset_index(drop=True)

    print("🔹 Most Frequently Contacted RNA Atoms by UNK:")
    display(df.head(30))
    df.to_csv(f"{out_prefix}ligand_rna_atom_contact_frequencies.csv", index=False)
    print(f"✅ Saved to '{out_prefix}ligand_rna_atom_contact_frequencies.csv'")


# 10. Carboxyl - OP1/2 P binding checker

def run_carboxylate_OP_checker(
    traj_file, top_file, ligand_resname="UNK", out_prefix="carboxylate_OP_"
):
    """
    Checks and visualizes ligand carboxylate O contacts with RNA OP1/OP2.
    Saves heatmap, barplot, and prints top contacts.
    """
    import mdtraj as md
    import jax.numpy as jnp
    import numpy as np
    import matplotlib.pyplot as plt
    import seaborn as sns
    from tqdm import tqdm

    traj = md.load_dcd(traj_file, top=top_file)
    atoms = list(traj.topology.atoms)

    # 1. Find ligand carboxylate O atoms
    carboxylate_O_names = {"O", "OXT", "O1", "O2"}
    ligand_O_idxs = [
        atom.index for atom in atoms
        if atom.residue.name.strip() == ligand_resname and atom.name.strip() in carboxylate_O_names
    ]
    if not ligand_O_idxs:
        ligand_idxs = [atom.index for atom in atoms if atom.residue.name.strip() == ligand_resname]
        ligand_O_idxs = [atom.index for atom in atoms if atom.residue.name.strip() == ligand_resname and atom.element.symbol == "O"]
        if not ligand_O_idxs:
            print("❗ No carboxylate O atoms found in the ligand. Analysis aborted.")
            return
        print("Auto-detected ligand O atoms:", ligand_O_idxs)

    print(f"Found ligand carboxylate O atoms (indices): {ligand_O_idxs}")

    # 2. Find all RNA OP1/OP2 atoms
    rna_bases = {"A", "U", "G", "C"}
    phos_names = {"OP1", "OP2"}
    rna_OP_idxs = []
    rna_OP_labels = []
    for atom in atoms:
        if atom.residue.name.strip() in rna_bases and atom.name.strip() in phos_names:
            rna_OP_idxs.append(atom.index)
            rna_OP_labels.append(f"{atom.residue.name.strip()}{atom.residue.resSeq}-{atom.name.strip()}")
    print(f"Found {len(rna_OP_idxs)} OP1/OP2 atoms in RNA backbone.")

    # 3. Extract coordinates
    coords = jnp.array(traj.xyz)
    ligand_coords = coords[:, ligand_O_idxs, :]
    rna_OP_coords = coords[:, rna_OP_idxs, :]
    n_frames = coords.shape[0]
    n_OP = len(rna_OP_idxs)
    n_ligand_O = len(ligand_O_idxs)

    # 4. Compute contacts
    threshold = 0.35  # nm
    contact_matrix = np.zeros((n_frames, n_OP), dtype=int)
    print("Computing contacts (carboxylate O ↔ OP1/OP2)...")
    for f in tqdm(range(n_frames)):
        for li in range(n_ligand_O):
            dists = jnp.linalg.norm(rna_OP_coords[f] - ligand_coords[f, li], axis=1)
            contact_matrix[f] = np.logical_or(contact_matrix[f], dists < threshold)

    # 5. Occupancy per OP
    occupancy = contact_matrix.sum(axis=0) / n_frames

    # 6. Save data for downstream analysis
    np.save(out_prefix + "contact_matrix.npy", contact_matrix)
    np.save(out_prefix + "occupancy.npy", occupancy)
    with open(out_prefix + "labels.txt", "w") as f:
        for label in rna_OP_labels:
            f.write(label + "\n")

    # 7. Plots
    plt.figure(figsize=(14, 3))
    sns.heatmap(contact_matrix.T, cmap="Blues", cbar=True, yticklabels=rna_OP_labels)
    plt.title("Ligand Carboxylate–RNA OP1/OP2 Contacts (framewise)")
    plt.xlabel("Frame")
    plt.ylabel("RNA OP1/OP2 Atom")
    plt.tight_layout()
    plt.savefig(out_prefix + "contact_heatmap.png", dpi=200)
    plt.show()

    plt.figure(figsize=(16, 4))
    sns.barplot(x=rna_OP_labels, y=occupancy)
    plt.xticks(rotation=90)
    plt.ylabel("Contact Occupancy (fraction)")
    plt.title("Occupancy of Ligand Carboxylate Contact with RNA OP1/OP2")
    plt.tight_layout()
    plt.savefig(out_prefix + "occupancy.png", dpi=200)
    plt.show()

    print(f"Saved: {out_prefix}contact_heatmap.png and {out_prefix}occupancy.png")

    # 8. Print top 10 contacts
    op_contact = list(zip(rna_OP_labels, occupancy))
    op_contact_sorted = sorted(op_contact, key=lambda x: x[1], reverse=True)
    print("\nTop 10 ligand–OP contacts by occupancy:")
    for label, occ in op_contact_sorted[:10]:
        print(f'{label}: {occ:.3f}')

    print(f"✅ Carboxylate–OP checker complete! (prefix '{out_prefix}')")

# 11. Backbone exposure by the ligand

def run_backbone_exposure_overlay(
    ligand_dcd="production_md.dcd",
    ligand_pdb="final_structure.pdb",
    control_dcd="production_md_no_ligand.dcd",
    control_pdb="final_structure_no_ligand.pdb",
    ligand_resname="UNK",
    topN=10,
    out_prefix="backbone_exposure_"
):
    """
    Compares RNA backbone exposure (proxy: neighbor count) with/without ligand.
    Plots and saves overlays, sorted by distance to ligand, and exports top-exposed atoms as a PDB.
    """
    import mdtraj as md
    import jax.numpy as jnp
    import numpy as np
    import matplotlib.pyplot as plt
    from tqdm import tqdm
    import pandas as pd
    import re

    # Helper for atom info
    def extract_atom_info(traj):
        return [
            {
                'index': atom.index,
                'resname': atom.residue.name.strip(),
                'resid': atom.residue.resSeq,
                'atomname': atom.name.strip()
            }
            for atom in traj.topology.atoms
        ]

    # --- Load holo ---
    traj_lig = md.load_dcd(ligand_dcd, top=ligand_pdb)
    atom_info_lig = extract_atom_info(traj_lig)
    coords_lig = jnp.array(traj_lig.xyz)
    backbone_names = {"OP1", "OP2", "P"}
    rna_resnames = {"A", "U", "G", "C"}
    backbone_lig = [
        i for i, atom in enumerate(atom_info_lig)
        if atom['resname'] in rna_resnames and atom['atomname'] in backbone_names
    ]
    all_lig = set(range(len(atom_info_lig)))
    nonrna_lig = list(all_lig - set([
        i for i, atom in enumerate(atom_info_lig)
        if atom['resname'] in rna_resnames
    ]))

    # --- Load apo ---
    traj_ctrl = md.load_dcd(control_dcd, top=control_pdb)
    atom_info_ctrl = extract_atom_info(traj_ctrl)
    coords_ctrl = jnp.array(traj_ctrl.xyz)
    backbone_ctrl = [
        i for i, atom in enumerate(atom_info_ctrl)
        if atom['resname'] in rna_resnames and atom['atomname'] in backbone_names
    ]
    all_ctrl = set(range(len(atom_info_ctrl)))
    nonrna_ctrl = list(all_ctrl - set([
        i for i, atom in enumerate(atom_info_ctrl)
        if atom['resname'] in rna_resnames
    ]))

    # --- Exposure calculation (same as your proxy) ---
    def exposure_count(coords, backbone_idxs, nonrna_idxs, cutoff=0.35):
        n_frames = coords.shape[0]
        n_backbone = len(backbone_idxs)
        counts = np.zeros((n_frames, n_backbone), dtype=int)
        for f in tqdm(range(n_frames), desc="Frames (Exposure)"):
            nonrna_pos = coords[f, nonrna_idxs, :]
            for k, bb_idx in enumerate(backbone_idxs):
                bb_pos = coords[f, bb_idx, :]
                dists = jnp.linalg.norm(nonrna_pos - bb_pos, axis=1)
                counts[f, k] = (dists < cutoff).sum()
        return counts

    counts_lig = exposure_count(coords_lig, backbone_lig, nonrna_lig)
    counts_ctrl = exposure_count(coords_ctrl, backbone_ctrl, nonrna_ctrl)

    mean_exp_lig = counts_lig.mean(axis=0)
    mean_exp_ctrl = counts_ctrl.mean(axis=0)

    # Label backbone atoms by (resname, resid, atomname)
    labels_lig = [
        f"{atom_info_lig[i]['resname']}{atom_info_lig[i]['resid']}-{atom_info_lig[i]['atomname']}"
        for i in backbone_lig
    ]
    labels_ctrl = [
        f"{atom_info_ctrl[i]['resname']}{atom_info_ctrl[i]['resid']}-{atom_info_ctrl[i]['atomname']}"
        for i in backbone_ctrl
    ]

    # Match by labels for comparison
    lig_map = dict(zip(labels_lig, mean_exp_lig))
    ctrl_map = dict(zip(labels_ctrl, mean_exp_ctrl))
    common_labels = sorted(set(labels_lig) & set(labels_ctrl))
    delta_exp = np.array([lig_map[l] - ctrl_map[l] for l in common_labels])

    # Save CSV with all data
    df_sasa = pd.DataFrame({
        "label": common_labels,
        "exp_lig": [lig_map[l] for l in common_labels],
        "exp_ctrl": [ctrl_map[l] for l in common_labels],
        "delta_exp": delta_exp
    })
    df_sasa.to_csv(out_prefix + "sasa.csv", index=False)

    # --- Ligand centroid & backbone distances ---
    ligand_atoms = [i for i, a in enumerate(atom_info_lig) if a['resname'] == ligand_resname]
    if len(ligand_atoms) == 0:
        print("❗ No ligand atoms found. Check ligand_resname!")
        return
    ligand_coords = coords_lig[:, ligand_atoms, :]
    ligand_centroid = ligand_coords.mean(axis=(0, 1))
    bb_positions = np.array([
        coords_lig[:, idx, :].mean(axis=0) for idx in backbone_lig
    ])
    dist_to_ligand = np.linalg.norm(bb_positions - ligand_centroid, axis=1)

    # Reorder by distance for default plots
    order = np.argsort(dist_to_ligand)
    sorted_labels = [common_labels[i] for i in order]
    sorted_delta = delta_exp[order]
    sorted_dist = dist_to_ligand[order]

    # Save ordered CSV too
    df_sasa_sorted = pd.DataFrame({
        "label": sorted_labels,
        "delta_exp": sorted_delta,
        "dist_to_ligand": sorted_dist
    })
    df_sasa_sorted.to_csv(out_prefix + "delta_exposure_sasa_sorted.csv", index=False)

    # --- Plots ---
    plt.figure(figsize=(20,5))
    plt.bar(sorted_labels, sorted_delta)
    plt.axhline(0, color='k', linestyle='--', alpha=0.6)
    plt.xticks(rotation=90, fontsize=8)
    plt.ylabel("ΔExposure (ligand – control, lower = more exposed)")
    plt.title("Backbone Exposure Proxy (Contact Number): Ligand vs. Control")
    plt.tight_layout()
    plt.savefig(out_prefix + "delta_backbone_exposure_proxy.png", dpi=180)
    plt.show()

    plt.figure(figsize=(22,5))
    plt.bar(range(len(sorted_labels)), sorted_delta, tick_label=sorted_labels)
    plt.axhline(0, color='k', linestyle='--', alpha=0.6)
    plt.xticks(rotation=90, fontsize=8)
    plt.ylabel("ΔExposure (ligand – control)")
    plt.xlabel("Backbone atoms, sorted by distance from ligand centroid (closest → farthest)")
    plt.title("Backbone Exposure Change (ΔExposure) vs. Proximity to Ligand")
    plt.tight_layout()
    plt.savefig(out_prefix + "delta_exposure_sorted_by_distance.png", dpi=180)
    plt.show()

    plt.figure(figsize=(7,4))
    plt.scatter(sorted_dist, sorted_delta, c=sorted_delta, cmap='coolwarm')
    plt.axhline(0, color='k', linestyle='--', alpha=0.5)
    plt.xlabel("Distance to Ligand Centroid (nm)")
    plt.ylabel("ΔExposure (ligand – control)")
    plt.title("Exposure Change vs. Ligand Distance")
    plt.tight_layout()
    plt.savefig(out_prefix + "delta_exposure_vs_distance.png", dpi=180)
    plt.show()

    # --- Top N hotspots (most negative delta) ---
    df_hotspot = df_sasa_sorted.nsmallest(topN, 'delta_exp')
    print(f"\nTop {topN} most negatively exposed backbone atoms (hotspots):")
    print(df_hotspot)
    df_hotspot.to_csv(out_prefix + f"top{topN}_exposed.csv", index=False)

    # --- Export as PDB for ChimeraX ---
    try:
        from Bio.PDB import PDBParser, PDBIO, Select
        pdb = ligand_pdb
        # Get set of (resname, resid) for all top-exposed backbone atoms
        hotspot_residues = set()
        for label in df_hotspot['label']:
            match = re.match(r'([AUGC])(\d+)-(\w+)', label)
            if match:
                resname, resid, atomname = match.groups()
                hotspot_residues.add((resname, int(resid)))
        parser = PDBParser(QUIET=True)
        structure = parser.get_structure("RNA", pdb)
        class ResidueSelect(Select):
            def accept_residue(self, residue):
                resname = residue.get_resname().strip()
                resid = residue.get_id()[1]
                return (resname, resid) in hotspot_residues
        io = PDBIO()
        io.set_structure(structure)
        io.save(out_prefix + "top_exposed_residues.pdb", ResidueSelect())
        print(f"Saved: {out_prefix}top_exposed_residues.pdb")
    except Exception as e:
        print("❗ Could not write PDB for hotspots:", e)

    print("✅ Backbone exposure checker complete!")

# 12. π–π stacking analysis

def run_pi_pi_stacking(traj_file, top_file, ligand_ring_defs, ligand_resname="UNK", out_prefix="pi_pi_"):
    import mdtraj as md
    import jax.numpy as jnp
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    from tqdm import tqdm

    traj = md.load_dcd(traj_file, top=top_file)
    coords = jnp.array(traj.xyz)
    n_frames = coords.shape[0]
    top = traj.topology

    # Load ring mapping
    hexagon_ring_atoms = {"N1", "C2", "N3", "C4", "C5", "C6"}
    hex_ring_atom_map = []
    for res in top.residues:
        base = res.name.strip()
        if base in {"A", "G", "C", "U"}:
            for atom in res.atoms:
                if atom.name.strip() in hexagon_ring_atoms:
                    hex_ring_atom_map.append((res.index, base, atom.name.strip(), atom.index))
    rna_ring_df = pd.DataFrame(hex_ring_atom_map, columns=["res_index", "base", "atom", "atom_index"])

    # RNA base hexagon rings (list of 6 atom indices per base)
    rna_base_ring_idxs = []
    rna_base_ring_labels = []
    rna_base_pdbinfo = []
    for res_index in rna_ring_df["res_index"].unique():
        subdf = rna_ring_df[rna_ring_df["res_index"] == res_index]
        if len(subdf) == 6:
            idxs = subdf.sort_values("atom")["atom_index"].tolist()
            atom0 = top.atom(int(subdf.iloc[0]["atom_index"]))
            chain = getattr(atom0.residue.chain, "id", None) or getattr(atom0.residue.chain, "name", None) or "A"
            base = atom0.residue.name
            resSeq = atom0.residue.resSeq
            label = f"{base}{resSeq}"
            rna_base_ring_idxs.append(idxs)
            rna_base_ring_labels.append(label)
            rna_base_pdbinfo.append((chain, base, resSeq, label))

    # --- Ligand ring indices
    ligand_atoms = {atom.name.strip(): atom.index for atom in top.atoms if atom.residue.name.strip() == ligand_resname}
    ligand_ring_idxs = []
    for i, ring in enumerate(ligand_ring_defs):
        try:
            idxs = [ligand_atoms[a] for a in ring]
        except KeyError as e:
            print(f"❗ Ligand atom name not found: {e}")
            return
        ligand_ring_idxs.append(idxs)

    # Centroid/normal helper
    def get_centroid_and_normal(ring_coords):
        centroid = jnp.mean(ring_coords, axis=0)
        v1 = ring_coords[1] - ring_coords[0]
        v2 = ring_coords[2] - ring_coords[0]
        normal = jnp.cross(v1, v2)
        normal = normal / jnp.linalg.norm(normal)
        return centroid, normal

    distance_cutoff = 0.6  # nm
    angle_cutoff_deg = 30

    stack_events = []
    for f in tqdm(range(n_frames), desc="Frames (π–π stacking)"):
        for lidx, lring_idxs in enumerate(ligand_ring_idxs):
            l_ring_coords = coords[f, lring_idxs, :]
            l_centroid, l_normal = get_centroid_and_normal(l_ring_coords)
            for ridx, rring_idxs in enumerate(rna_base_ring_idxs):
                r_ring_coords = coords[f, rring_idxs, :]
                r_centroid, r_normal = get_centroid_and_normal(r_ring_coords)
                dist = jnp.linalg.norm(l_centroid - r_centroid)
                cos_angle = jnp.dot(l_normal, r_normal)
                angle = jnp.degrees(jnp.arccos(jnp.clip(cos_angle, -1, 1)))
                if dist <= distance_cutoff and (angle <= angle_cutoff_deg or angle >= 180 - angle_cutoff_deg):
                    chain, base, resSeq, label = rna_base_pdbinfo[ridx]
                    stack_events.append((f, lidx, ridx, dist, angle, chain, base, resSeq, label))

    stack_df = pd.DataFrame(stack_events, columns=["frame", "ligand_ring", "rna_base_index", "distance", "angle", "chain", "base", "resSeq", "pdb_label"])
    stack_df.to_csv(out_prefix + "stacking_events_PDBlabels.csv", index=False)
    print(f"✅ Saved {out_prefix}stacking_events_PDBlabels.csv with robust PDB labels.")

    if not stack_df.empty:
        counts = stack_df["pdb_label"].value_counts().sort_index()
        plt.figure(figsize=(14,4))
        plt.bar(counts.index, counts.values)
        plt.xticks(rotation=90)
        plt.ylabel("Stacking Events")
        plt.title("π–π Stacking Events (Ligand–RNA Hexagon Rings, PDB numbering)")
        plt.tight_layout()
        plt.savefig(out_prefix + "stacking_events_PDBlabels_barplot.png", dpi=150)
        plt.show()
    else:
        print("No π–π stacking events detected with current cutoffs.")





#### *** After aromatic ring definition (If necessary), and running the setup cell above, run this cell below. Toggle panel appears for selecting analyses. Some analyses may take a few minutes. Please be patient.***

In [ ]:
#@title [UI] Run for Analysis Toggle Panel { display-mode: "form" }

import ipywidgets as widgets
from IPython.display import display, clear_output

# Each (label, checkbox) pair for clarity
labels_and_checkboxes = [
    # --- Protein/RNA common analyses ---
    (widgets.Label("Global RMSD (Overlay)"), widgets.Checkbox(value=False, description="Run")),
    (widgets.Label("Global RMSF (Overlay)"), widgets.Checkbox(value=False, description="Run")),
    (widgets.Label("Ligand RMSD (Holo only)"), widgets.Checkbox(value=False, description="Run")),
    (widgets.Label("Hydration Shell (Holo only)"), widgets.Checkbox(value=False, description="Run")),
    (widgets.Label("H-Bond Analysis RNA (Holo only, all outputs)"), widgets.Checkbox(value=False, description="Run")),
    (widgets.Label("H-Bond Analysis Protein (Holo only, all outputs)"), widgets.Checkbox(value=False, description="Run")),
    (widgets.Label("Residue-Ligand Contacts RNA (Holo only, all outputs)"), widgets.Checkbox(value=False, description="Run")),
    (widgets.Label("Residue-Ligand Contacts Protein (Holo only, all outputs)"), widgets.Checkbox(value=False, description="Run")),
    (widgets.Label("PCA + FEL Overlay"), widgets.Checkbox(value=False, description="Run")),
    # --- RNA specialized analyses ---
    (widgets.Label("Watson–Crick Pair Checker (Overlay apo/holo)"), widgets.Checkbox(value=False, description="Run")),
    (widgets.Label("Ligand–RNA Atom Contact Frequencies (after H-bonds, RNA only)"), widgets.Checkbox(value=False, description="Run")),
    (widgets.Label("Carboxylate–OP1/OP2 Checker (RNA only, ligand required)"), widgets.Checkbox(value=False, description="Run")),
    (widgets.Label("Backbone Exposure Change (allosteric, Δexposure, RNA only)"), widgets.Checkbox(value=False, description="Run")),
    (widgets.Label("π–π Stacking (ligand–RNA base rings, RNA only, ligand ring input required)"), widgets.Checkbox(value=False, description="Run")),
]

# Unpack for easier reference later
rmsd_checkbox         = labels_and_checkboxes[0][1]
rmsf_checkbox         = labels_and_checkboxes[1][1]
ligand_rmsd_checkbox  = labels_and_checkboxes[2][1]
hydration_shell_checkbox = labels_and_checkboxes[3][1]
hbond_checkbox_rna    = labels_and_checkboxes[4][1]
hbond_checkbox_prot   = labels_and_checkboxes[5][1]
contact_checkbox_rna  = labels_and_checkboxes[6][1]
contact_checkbox_prot = labels_and_checkboxes[7][1]
pca_checkbox          = labels_and_checkboxes[8][1]

wc_pair_checker_checkbox = labels_and_checkboxes[9][1]
ligand_rna_atom_contact_checkbox = labels_and_checkboxes[10][1]
carboxylate_op_checkbox = labels_and_checkboxes[11][1]
backbone_exposure_checkbox = labels_and_checkboxes[12][1]
pi_pi_checkbox = labels_and_checkboxes[13][1]

widget_rows = [widgets.HBox([label, cb]) for label, cb in labels_and_checkboxes]
run_button = widgets.Button(description="Run Selected Analyses", button_style='success', layout=widgets.Layout(width='250px'))
output = widgets.Output()
display(widgets.VBox(widget_rows + [run_button, output]))

def on_run_button_clicked(b):
    with output:
        clear_output()
        # Apo/No ligand files
        traj_apo = "production_md_no_ligand.dcd"
        top_apo = "final_structure_no_ligand.pdb"
        # Holo/Ligand files
        traj_holo = "production_md.dcd"
        top_holo = "final_structure.pdb"
        ligand_resname = "UNK"  # Update as needed

        # --- Common analyses ---
        if rmsd_checkbox.value:
            run_rmsd_overlay(traj_apo, top_apo, traj_holo, top_holo)
        if rmsf_checkbox.value:
            run_rmsf_overlay(traj_apo, top_apo, traj_holo, top_holo)
        if ligand_rmsd_checkbox.value:
            run_ligand_rmsd(traj_holo, top_holo, ligand_resname)
        if hydration_shell_checkbox.value:
            run_hydration_shell(traj_holo, top_holo, ligand_resname)
        if hbond_checkbox_rna.value:
            run_hbond_analysis_rna(traj_holo, top_holo, ligand_resname=ligand_resname)
        if hbond_checkbox_prot.value:
            run_hbond_analysis_protein(traj_holo, top_holo, ligand_resname=ligand_resname)
        if contact_checkbox_rna.value:
            run_contact_analysis_rna(traj_holo, top_holo, ligand_resname=ligand_resname)
        if contact_checkbox_prot.value:
            run_contact_analysis_protein(traj_holo, top_holo, ligand_resname=ligand_resname)
        if pca_checkbox.value:
            run_pca_fel_overlay(traj_apo, top_apo, traj_holo, top_holo)

        # --- RNA specialized analyses ---
        if wc_pair_checker_checkbox.value:
            run_wc_pair_checker_overlay(traj_apo, top_apo, traj_holo, top_holo)
        if ligand_rna_atom_contact_checkbox.value:
            run_ligand_rna_atom_contact_frequencies_safe()
        if carboxylate_op_checkbox.value:
            run_carboxylate_OP_checker(traj_holo, top_holo, ligand_resname)
        if backbone_exposure_checkbox.value:
            run_backbone_exposure_overlay(ligand_dcd=traj_holo, ligand_pdb=top_holo,
                                          control_dcd=traj_apo, control_pdb=top_apo,
                                          ligand_resname=ligand_resname)
        if pi_pi_checkbox.value:
            try:
                run_pi_pi_stacking(traj_holo, top_holo, ligand_ring_defs, ligand_resname)
            except NameError:
                print("❗ Please define 'ligand_ring_defs' in the setup cell above before running π–π stacking.")

        print("✅ All selected analyses complete!")

run_button.on_click(on_run_button_clicked)


## DTI with DeepPurpose

In [ ]:
# Dependency version check

import sys
import torch
import torchvision
import torchaudio
import transformers
import importlib.util

print("🔍 Python Version:", sys.version)
print("🧠 Torch Version:", torch.__version__)
print("🖼️ Torchvision Version:", torchvision.__version__)
print("🎵 Torchaudio Version:", torchaudio.__version__)
print("🚀 Transformers Version:", transformers.__version__)
print("⚡ CUDA Available:", torch.cuda.is_available())
print("🧪 CUDA Device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "N/A")

# Check if problematic modules are importable
def check_module(name):
    if importlib.util.find_spec(name):
        print(f"✅ {name} is available")
    else:
        print(f"❌ {name} NOT found")

print("\n🔍 Checking specific modules...")
check_module("torch.sparse._triton_ops_meta")
check_module("triton")
check_module("transformers.models.t5.modeling_t5")
check_module("transformers.models.t5")

# Inspect if T5EncoderModel is present
try:
    from transformers import T5EncoderModel
    print("\n✅ T5EncoderModel is importable")
except ImportError as e:
    print("\n❌ T5EncoderModel ImportError:", str(e))


🔍 Python Version: 3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0]
🧠 Torch Version: 2.6.0+cu124
🖼️ Torchvision Version: 0.21.0+cu124
🎵 Torchaudio Version: 2.6.0+cu124
🚀 Transformers Version: 4.52.4
⚡ CUDA Available: True
🧪 CUDA Device: NVIDIA A100-SXM4-40GB

🔍 Checking specific modules...
✅ torch.sparse._triton_ops_meta is available
✅ triton is available
✅ transformers.models.t5.modeling_t5 is available
✅ transformers.models.t5 is available

✅ T5EncoderModel is importable


In [ ]:
# Step 7: Deeppurpose
!conda run -n deeppurpose-md-env python scripts/7_deeppurpose_training.py

✅ Cleaned TSV saved as cleaned_bindingdb.tsv
✅ Cleaned and saved 14493 compound-target pairs to processed_bindingdb.tsv
✅ Loaded 14493 cleaned entries from processed_bindingdb.tsv
🔎 Unique protein sequences: 137
✅ Embeddings saved to protein_embeddings.pkl
✅ Merged ProtTrans embeddings with dataset → embedded_bindingdb.pkl
✅ Saved protein frequency table to top_protein_targets.csv
Figure(1000x400)
✅ Final dataset shape: (813, 5)
🧪 Original dataset size: 813
✅ After SMILES cleaning: 813
✅ Injected true 1024-dim ProtTrans embeddings → 813 entries
✅ PCA reduced ProtTrans shape: (813, 512)
🧊 PCA model saved to pca_model.pkl
Drug Target Interaction Prediction Mode...
in total: 1049 drug-target pairs
encoding drug...
unique drugs: 652
encoding protein...
unique target sequence: 55
✅ It's read.
✅ First target_encoding sample: [ 6.41156793e-01 -7.40467787e-01 -3.96605730e-02 -6.16653264e-03
  2.07237303e-02  2.30188668e-03  1.73478127e-02  1.25811026e-02
  9.50679183e-03  1.05562806e-02  2.296

In [ ]:
# Step 8: Deeppurpose prediction
!conda run -n deeppurpose-md-env python scripts/8_deeppurpose_prediction.py

✅ Parsed 12 sequences from protein.faa
✅ Combined total: 14 sequences → combined_fasta_targets.csv
🧬 FASTA entries found in protein.faa:
- YP_009724389.1 | YP_009724389.1 ORF1ab polyprotein [Severe acute respiratory syndrome coronavirus 2]
- YP_009724390.1 | YP_009724390.1 surface glycoprotein [Severe acute respiratory syndrome coronavirus 2]
- YP_009724391.1 | YP_009724391.1 ORF3a protein [Severe acute respiratory syndrome coronavirus 2]
- YP_009724392.1 | YP_009724392.1 envelope protein [Severe acute respiratory syndrome coronavirus 2]
- YP_009724393.1 | YP_009724393.1 membrane glycoprotein [Severe acute respiratory syndrome coronavirus 2]
- YP_009724394.1 | YP_009724394.1 ORF6 protein [Severe acute respiratory syndrome coronavirus 2]
- YP_009724395.1 | YP_009724395.1 ORF7a protein [Severe acute respiratory syndrome coronavirus 2]
- YP_009724396.1 | YP_009724396.1 ORF8 protein [Severe acute respiratory syndrome coronavirus 2]
- YP_009724397.2 | YP_009724397.2 nucleocapsid phosphoprot